In [1]:
import os
import sys
import torch
import numpy as np
import potpourri3d as pp3d
sys.path.append(os.path.join(os.getcwd(), "../../src/"))  # add the path to the DiffusionNet src
import diffusion_net

In [6]:
# Set device
device = torch.device('cpu')

# Path to the pretrained model
pretrain_path = "data/saved_models/sal_model.pth"

# File paths
root_dir = "data/saliency_data/"  # Change this to the actual root directory of your dataset
file_name = "bimba.ply"  # Change this to the actual mesh file name
label_file = "bimba.val"  # Change this to the actual label file name

In [3]:
# Load the mesh and labels
verts, faces = pp3d.read_mesh(os.path.join(root_dir, "ply", file_name))
labels = np.loadtxt(os.path.join(root_dir, "labels", label_file)).astype(int) + 1  # shift -1 --> 0

verts = torch.tensor(verts).float()
faces = torch.tensor(faces)
labels = torch.tensor(labels)

# Normalize positions
verts = diffusion_net.geometry.normalize_positions(verts)

In [4]:
# Compute operators
k_eig = 128  # Change this if needed
verts_list, faces_list = [verts], [faces]
frames, mass, L, evals, evecs, gradX, gradY = diffusion_net.geometry.get_all_operators(verts_list, faces_list, k_eig=k_eig)

# Convert to tensors
frames = torch.tensor(frames[0]).to(device)
mass = torch.tensor(mass[0]).to(device)
L = torch.tensor(L[0]).to(device)
evals = torch.tensor(evals[0]).to(device)
evecs = torch.tensor(evecs[0]).to(device)
gradX = torch.tensor(gradX[0]).to(device)
gradY = torch.tensor(gradY[0]).to(device)
labels = labels.to(device)

get_all_operators() processing 0 / 1 0.000%


C:\Users\ZAKARIA\AppData\Local\Temp\ipykernel_6636\4048726310.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frames = torch.tensor(frames[0]).to(device)
C:\Users\ZAKARIA\AppData\Local\Temp\ipykernel_6636\4048726310.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mass = torch.tensor(mass[0]).to(device)
C:\Users\ZAKARIA\AppData\Local\Temp\ipykernel_6636\4048726310.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L = torch.tensor(L[0]).to(device)
C:\Users\ZAKARIA\AppData\Local\Temp\ipykernel_6636\4048726310.py:10: UserWarning: To

In [7]:
# Load the model
input_features = "hks"  # Change this if using 'hks'
C_in = {'xyz': 3, 'hks': 16}[input_features]
n_class = 1

model = diffusion_net.layers.DiffusionNet(
    C_in=C_in,
    C_out=n_class,
    C_width=128,
    N_block=4,
    last_activation=torch.nn.functional.sigmoid,
    outputs_at='vertices',
    dropout=True
).to(device)

model.load_state_dict(torch.load(pretrain_path, map_location=device))
model.eval()

DiffusionNet(
  (first_lin): Linear(in_features=16, out_features=128, bias=True)
  (last_lin): Linear(in_features=128, out_features=1, bias=True)
  (block_0): DiffusionNetBlock(
    (diffusion): LearnedTimeDiffusion()
    (gradient_features): SpatialGradientFeatures(
      (A_re): Linear(in_features=128, out_features=128, bias=False)
      (A_im): Linear(in_features=128, out_features=128, bias=False)
    )
    (mlp): MiniMLP(
      (miniMLP_mlp_layer_000): Linear(in_features=384, out_features=128, bias=True)
      (miniMLP_mlp_act_000): ReLU()
      (miniMLP_mlp_layer_dropout_001): Dropout(p=0.5, inplace=False)
      (miniMLP_mlp_layer_001): Linear(in_features=128, out_features=128, bias=True)
      (miniMLP_mlp_act_001): ReLU()
      (miniMLP_mlp_layer_dropout_002): Dropout(p=0.5, inplace=False)
      (miniMLP_mlp_layer_002): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (block_1): DiffusionNetBlock(
    (diffusion): LearnedTimeDiffusion()
    (gradient_features): S

In [8]:
# Prepare input features
if input_features == 'xyz':
    features = verts.to(device)
elif input_features == 'hks':
    features = diffusion_net.geometry.compute_hks_autoscale(evals, evecs, 16).to(device)


In [9]:
# Make predictions
with torch.no_grad():
    preds = model(features, mass, L=L, evals=evals, evecs=evecs, gradX=gradX, gradY=gradY)
    preds = preds.squeeze().cpu().numpy()

print("Predictions:", preds)

Predictions: [1. 1. 1. ... 1. 1. 1.]


c:\Users\ZAKARIA\.conda\envs\diffusion_net\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
verts.shape

torch.Size([50000, 3])